# Backend Functions Testing Notebook

This notebook tests all individual functions and tools in the backend directory (WITHOUT agent integration).

## Modules being tested:
1. **schema.py** - Pydantic schemas for Resume and Job data structures
2. **manual_parser.py** - Manual parsing of job search results
3. **cv_parser.py** - CV/Resume parsing core functions
4. **google_cse_linkedin_search.py** - Google CSE LinkedIn search functions
5. **cv_parser_tool.py** - CV parser tool functions
6. **linkedin_job_search_tool.py** - LinkedIn job search tool
7. **job_search_from_cv_tool.py** - Smart CV-based job search tool

**Note:** For agent testing (create_linkedin_job_agent + agent.invoke), see `agent_tools_test.ipynb`

In [ ]:
# Import required libraries
import sys
import os
import json
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

# Add backend to Python path
backend_path = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if backend_path not in sys.path:
    sys.path.append(backend_path)

print(f"Backend tools path: {backend_path}")
print(f"Current working directory: {os.getcwd()}")

# Import model configuration from config
from config import MODEL_NAME
print(f"Using model: {MODEL_NAME}")


## 1. Test Schema Validation

In [ ]:
# Test schema.py - Pydantic schemas
try:
    from schema import (
        ResumeSchema, JobSchema, ContactInfo, Experience, Education, 
        Skill, Certification, Project, Language, CompanyInfo, 
        LinkedInJobSearchInput, CVParseInput, JobSearchFromCVInput
    )
    print("✅ Successfully imported all schemas including tool input schemas")
    print("Available schemas:")
    print("- Data schemas: ResumeSchema, JobSchema, ContactInfo, Experience, etc.")
    print("- Tool input schemas: LinkedInJobSearchInput, CVParseInput, JobSearchFromCVInput")
except ImportError as e:
    print(f"❌ Schema import error: {e}")


In [ ]:
# Test ContactInfo schema
print("=== Testing ContactInfo Schema ===")
try:
    contact = ContactInfo(
        email="test@example.com",
        phone="+1-234-567-8900",
        city="San Francisco",
        state="California",
        country="USA",
        linkedin="https://linkedin.com/in/testuser",
        github="https://github.com/testuser"
    )
    print("✅ ContactInfo validation successful")
    pprint(contact.dict())
except Exception as e:
    print(f"❌ ContactInfo validation error: {e}")


In [ ]:
# Test ResumeSchema with sample data
print("=== Testing ResumeSchema ===")
try:
    sample_resume = ResumeSchema(
        full_name="John Doe",
        professional_title="Software Engineer",
        summary="Experienced software developer with 5+ years in web development",
        total_years_experience="5",
        preferred_work_type="Remote"
    )
    print("✅ ResumeSchema validation successful")
    print(f"Name: {sample_resume.full_name}")
    print(f"Title: {sample_resume.professional_title}")
    print(f"Experience: {sample_resume.total_years_experience} years")
except Exception as e:
    print(f"❌ ResumeSchema validation error: {e}")


In [ ]:
# Test JobSchema with sample data
print("=== Testing JobSchema ===")
try:
    sample_job = JobSchema(
        title="Senior Python Developer",
        seniority_level="Senior",
        location="San Francisco, CA",
        work_arrangement="Remote",
        job_type="Full-time",
        required_skills=["Python", "Django", "PostgreSQL"],
        technologies=["Docker", "AWS", "Redis"],
        salary_min="120000",
        salary_max="180000",
        salary_currency="USD"
    )
    print("✅ JobSchema validation successful")
    print(f"Title: {sample_job.title}")
    print(f"Location: {sample_job.location}")
    print(f"Skills: {sample_job.required_skills}")
    print(f"Salary: {sample_job.salary_min}-{sample_job.salary_max} {sample_job.salary_currency}")
except Exception as e:
    print(f"❌ JobSchema validation error: {e}")


In [ ]:
# Test LinkedInJobSearchInput schema
print("=== Testing LinkedInJobSearchInput Schema ===")
try:
    search_input = LinkedInJobSearchInput(
        keyword="Python Developer",
        location="Remote",
        job_type="full-time",
        experience_level="senior",
        num_results=5,
        parsing_method="llm",
        model_name=MODEL_NAME
    )
    print("✅ LinkedInJobSearchInput validation successful")
    print(f"Keyword: {search_input.keyword}")
    print(f"Location: {search_input.location}")
    print(f"Model: {search_input.model_name}")
    print(f"Num results: {search_input.num_results}")
except Exception as e:
    print(f"❌ LinkedInJobSearchInput validation error: {e}")

print("\n=== Testing CVParseInput Schema ===")
try:
    cv_input = CVParseInput(
        content="Sample CV content here...",
        content_type="text",
        model_name=MODEL_NAME
    )
    print("✅ CVParseInput validation successful")
    print(f"Content type: {cv_input.content_type}")
    print(f"Model: {cv_input.model_name}")
    print(f"Content length: {len(cv_input.content)} characters")
except Exception as e:
    print(f"❌ CVParseInput validation error: {e}")

print("\n=== Testing JobSearchFromCVInput Schema ===")
try:
    cv_job_input = JobSearchFromCVInput(
        cv_content="Sample CV content for job search...",
        cv_content_type="text",
        location="San Francisco, CA",
        job_type="full-time",
        work_arrangement="remote",
        num_results=10,
        include_entry_level=False,
        model_name=MODEL_NAME
    )
    print("✅ JobSearchFromCVInput validation successful")
    print(f"CV content type: {cv_job_input.cv_content_type}")
    print(f"Location: {cv_job_input.location}")
    print(f"Work arrangement: {cv_job_input.work_arrangement}")
    print(f"Include entry level: {cv_job_input.include_entry_level}")
    print(f"Model: {cv_job_input.model_name}")
except Exception as e:
    print(f"❌ JobSearchFromCVInput validation error: {e}")


## 2. Test Manual Parser

In [ ]:
# Test manual_parser.py
try:
    from manual_parser import LinkedInJobManualParser
    print("✅ Successfully imported LinkedInJobManualParser")
except ImportError as e:
    print(f"❌ Manual parser import error: {e}")


In [ ]:
# Test LinkedInJobManualParser with sample data
print("=== Testing LinkedInJobManualParser ===")
try:
    parser = LinkedInJobManualParser()
    
    # Sample data similar to Google CSE results
    sample_search_data = {
        'items': [
            {
                'title': 'Senior Python Developer - Google - LinkedIn',
                'link': 'https://www.linkedin.com/jobs/view/1234567890',
                'snippet': 'Google is hiring a Senior Python Developer in San Francisco, CA. Full-time position with competitive salary $150,000-$200,000. Experience with Django, PostgreSQL required. Posted 2 days ago.'
            },
            {
                'title': 'Frontend Developer at Microsoft | LinkedIn',
                'link': 'https://www.linkedin.com/jobs/view/9876543210',
                'snippet': 'Microsoft seeks Frontend Developer for Remote position. React, TypeScript, Node.js experience required. Salary: $120,000-$160,000. Posted 1 week ago.'
            }
        ]
    }
    
    # Test parsing search results
    jobs = parser.parse_search_results(sample_search_data)
    print(f"✅ Parsed {len(jobs)} jobs successfully")
    
    for i, job in enumerate(jobs, 1):
        print(f"\n--- Job {i} ---")
        print(f"Title: {job.get('title', 'N/A')}")
        print(f"Company: {job.get('company', 'N/A')}")
        print(f"Location: {job.get('location', 'N/A')}")
        print(f"Job Type: {job.get('job_type', 'N/A')}")
        print(f"Salary: {job.get('salary', 'N/A')}")
        print(f"Posted: {job.get('posted_date', 'N/A')}")
        print(f"URL: {job.get('url', 'N/A')}")
        
except Exception as e:
    print(f"❌ Manual parser test error: {e}")


## 3. Test CV Parser

In [ ]:
# Test cv_parser.py
try:
    from cv_parser import CVParser, parse_resume_text, parse_resume_pdf
    print("✅ Successfully imported CV parser components")
except ImportError as e:
    print(f"❌ CV parser import error: {e}")

In [ ]:
# Test CV parsing with sample text
print("=== Testing CV Parsing from Text ===")

sample_resume_text = """
JOHN DOE
Software Engineer
Email: john.doe@email.com
Phone: +1-234-567-8900
Location: San Francisco, CA
LinkedIn: https://linkedin.com/in/johndoe

PROFESSIONAL SUMMARY
Experienced Full-Stack Software Engineer with 5+ years of experience developing scalable web applications using Python, JavaScript, and modern frameworks.

TECHNICAL SKILLS
• Programming Languages: Python, JavaScript, TypeScript, Java
• Web Frameworks: Django, Flask, React, Node.js
• Databases: PostgreSQL, MongoDB, Redis
• Cloud Platforms: AWS, Google Cloud Platform

WORK EXPERIENCE

Senior Software Engineer | Tech Corp | Jan 2021 - Present
San Francisco, CA
• Led development of microservices architecture serving 1M+ users
• Implemented CI/CD pipelines reducing deployment time by 60%
• Technologies: Python, Django, PostgreSQL, AWS, Docker

EDUCATION
Bachelor of Science in Computer Science
University of California, Berkeley | 2015 - 2019
GPA: 3.8/4.0

CERTIFICATIONS
• AWS Certified Solutions Architect (2022)
• Google Cloud Professional Developer (2021)
"""

try:
    # Test with utility function using configured model
    result = parse_resume_text(sample_resume_text, model_name=MODEL_NAME)
    
    print(f"Success: {result.get('success', False)}")
    
    if result.get('success'):
        data = result.get('data', {})
        print(f"\n--- Parsed Resume Data ---")
        print(f"Name: {data.get('full_name', 'N/A')}")
        print(f"Title: {data.get('professional_title', 'N/A')}")
        print(f"Email: {data.get('contact_info', {}).get('email', 'N/A')}")
        print(f"Experience Years: {data.get('total_years_experience', 'N/A')}")
        print(f"Number of Experiences: {len(data.get('experiences', []))}")
        print(f"Number of Skills: {len(data.get('technical_skills', []))}")
        print(f"Education: {len(data.get('education', []))} entries")
        print(f"Parsing method: {result.get('parsing_method', 'N/A')}")
        print(f"Model used: {MODEL_NAME}")
    else:
        print(f"Error: {result.get('error', 'Unknown error')}")
        
except Exception as e:
    print(f"❌ CV parsing test error: {e}")

## 4. Test Google CSE LinkedIn Search

In [ ]:
# Test google_cse_linkedin_search.py
try:
    from google_cse_linkedin_search import GoogleCSELinkedInSearcher
    print("✅ Successfully imported GoogleCSELinkedInSearcher")
except ImportError as e:
    print(f"❌ Google CSE search import error: {e}")

# Test GoogleCSELinkedInSearcher initialization
print("=== Testing GoogleCSELinkedInSearcher Initialization ===")
try:
    # Note: API keys in .env file are needed for real testing
    # This is just testing initialization with dummy values
    dummy_api_key = "dummy_api_key"
    dummy_search_engine_id = "dummy_search_engine_id"
    
    searcher = GoogleCSELinkedInSearcher(
        api_key=dummy_api_key,
        search_engine_id=dummy_search_engine_id,
        model_name=MODEL_NAME
    )
    
    print(f"✅ GoogleCSELinkedInSearcher initialized")
    print(f"Model name: {searcher.model_name}")
    print(f"LLM available: {searcher.llm_available}")
    print(f"Manual parser available: {searcher.manual_parser is not None}")
    print(f"Using model: {MODEL_NAME}")
    
except Exception as e:
    print(f"❌ GoogleCSELinkedInSearcher initialization error: {e}")


## 5. Test CV Parser Tool Functions

In [ ]:
# Test cv_parser_tool.py functions
try:
    from cv_parser_tool import parse_cv_content, extract_pdf_text
    print("✅ Successfully imported CV parser tool functions")
    print("Available functions:")
    print(f"- parse_cv_content: {parse_cv_content}")
    print(f"- extract_pdf_text: {extract_pdf_text}")
except ImportError as e:
    print(f"❌ CV parser tool import error: {e}")

# Test parse_cv_content function with sample text
print("\n=== Testing parse_cv_content Function ===")
sample_cv = """
John Smith
Senior Software Developer

Contact:
Email: john.smith@example.com
Phone: +1-555-123-4567
Location: Seattle, WA

Experience:
Senior Developer at TechCorp (2020-Present)
- Python, Django, PostgreSQL
- Led team of 4 developers
- Built microservices architecture

Education:
BS Computer Science, University of Washington (2016-2020)
"""

try:
    # Test function directly (will fail without LLM setup, but tests interface)
    result = parse_cv_content.func(
        content=sample_cv,
        content_type="text",
        model_name=MODEL_NAME
    )
    
    print(f"✅ Function called successfully")
    print(f"Success: {result.get('success', False)}")
    
    if result.get('success'):
        data = result.get('data', {})
        print(f"Parsed name: {data.get('full_name', 'N/A')}")
        print(f"Professional title: {data.get('professional_title', 'N/A')}")
        print(f"Email: {data.get('contact_info', {}).get('email', 'N/A')}")
    else:
        error = result.get('error', '')
        if 'llm' in error.lower() or 'model' in error.lower() or 'groq' in error.lower():
            print("✅ Expected error: LLM setup required (normal in test environment)")
        else:
            print(f"Error: {error}")
        
except Exception as e:
    print(f"❌ parse_cv_content test error: {e}")

# Test extract_pdf_text function
print("\n=== Testing extract_pdf_text Function ===")
try:
    # Test with non-existent file to test error handling
    result = extract_pdf_text.func(pdf_path="non_existent_file.pdf")
    
    print(f"✅ Function called successfully")
    print(f"Success: {result.get('success', False)}")
    
    if not result.get('success'):
        error = result.get('error', '')
        if 'not found' in error.lower() or 'pdf' in error.lower():
            print("✅ Expected error: File not found (correct error handling)")
        else:
            print(f"Error: {error}")
    else:
        print(f"Unexpected success for non-existent file")
        
except Exception as e:
    print(f"❌ extract_pdf_text test error: {e}")


## 6. Test LinkedIn Job Search Tool Functions

In [ ]:
# Test linkedin_job_search_tool.py functions
try:
    from linkedin_job_search_tool import search_linkedin_jobs
    print("✅ Successfully imported LinkedIn job search tool")
    print(f"search_linkedin_jobs function: {search_linkedin_jobs}")
except ImportError as e:
    print(f"❌ LinkedIn job search tool import error: {e}")

# Test search_linkedin_jobs function
print("\n=== Testing search_linkedin_jobs Function ===")
try:
    # Test function call (will fail without API key, but tests interface)
    result = search_linkedin_jobs.func(
        keyword="Python Developer",
        location="Remote",
        num_results=3,
        parsing_method="manual",  # Use manual to avoid LLM dependency
        model_name=MODEL_NAME
    )
    
    print(f"✅ Function called successfully")
    print(f"Success: {result.get('success', False)}")
    
    if not result.get('success'):
        error = result.get('error', '')
        if 'credentials' in error.lower() or 'api' in error.lower():
            print("✅ Expected error: Missing API credentials (normal in test environment)")
        else:
            print(f"Error: {error}")
    else:
        jobs = result.get('jobs', [])
        print(f"Jobs found: {len(jobs)}")
        if jobs:
            print(f"First job title: {jobs[0].get('title', 'N/A')}")
        
except Exception as e:
    print(f"❌ search_linkedin_jobs test error: {e}")

# Test function with various parameters
print("\n=== Testing Function Parameters ===")
test_params = [
    {
        "keyword": "Software Engineer",
        "location": "San Francisco, CA",
        "experience_level": "senior",
        "job_type": "full-time"
    },
    {
        "keyword": "Data Scientist", 
        "work_arrangement": "remote",
        "num_results": 5
    },
    {
        "keyword": "Frontend Developer",
        "company": "Google",
        "industry": "technology"
    }
]

for i, params in enumerate(test_params, 1):
    print(f"\n--- Test {i}: {params['keyword']} ---")
    try:
        # Add default params
        params.update({
            "parsing_method": "manual",
            "model_name": MODEL_NAME
        })
        
        result = search_linkedin_jobs.func(**params)
        print(f"✅ Parameters accepted")
        print(f"Success: {result.get('success', False)}")
        
        if not result.get('success'):
            error = result.get('error', '')
            if 'api' in error.lower() or 'credential' in error.lower():
                print("✅ Expected API error")
            else:
                print(f"Error: {error[:100]}...")
                
    except Exception as e:
        print(f"❌ Parameter test {i} error: {e}")


## 7. Test Job Search from CV Tool Functions

In [ ]:
# Test job_search_from_cv_tool.py functions
try:
    from job_search_from_cv_tool import search_jobs_from_cv
    print("✅ Successfully imported job search from CV tool")
    print(f"search_jobs_from_cv function: {search_jobs_from_cv}")
except ImportError as e:
    print(f"❌ Job search from CV tool import error: {e}")

# Sample CV for testing
sample_cv_for_search = """
Alice Johnson
Data Scientist

Contact Information:
Email: alice.johnson@email.com
Phone: +1-555-987-6543
Location: New York, NY

Professional Summary:
Experienced data scientist with 4+ years in machine learning and analytics.
Specialized in Python, R, and cloud platforms.

Work Experience:
Data Scientist | DataCorp Inc. | 2020 - Present
New York, NY
- Built ML models for customer segmentation using Python and scikit-learn
- Developed data pipelines with Apache Spark and AWS
- Led analytics projects for 500k+ user base

Technical Skills:
- Programming: Python, R, SQL, JavaScript  
- ML/AI: scikit-learn, TensorFlow, PyTorch
- Cloud: AWS, GCP, Azure
- Databases: PostgreSQL, MongoDB

Education:
MS Data Science | Columbia University | 2018-2020
"""

# Test search_jobs_from_cv function
print("\n=== Testing search_jobs_from_cv Function ===")
try:
    # Test function call (will fail without API keys, but tests interface)
    result = search_jobs_from_cv.func(
        cv_content=sample_cv_for_search,
        cv_content_type="text",
        location="",  # Let it use CV location
        num_results=3,
        include_entry_level=False,
        model_name=MODEL_NAME
    )
    
    print(f"✅ Function called successfully")
    print(f"Success: {result.get('success', False)}")
    
    if result.get('success'):
        print(f"CV analysis success: {result.get('cv_analysis', {}).get('success', False)}")
        print(f"Job search success: {result.get('job_search_success', False)}")
        print(f"Jobs found: {result.get('jobs_found', 0)}")
        
        # Show strategy if available
        strategy = result.get('search_strategy', {})
        if strategy:
            print(f"Generated keyword: {strategy.get('keyword', 'N/A')}")
            print(f"Generated location: {strategy.get('location', 'N/A')}")
            print(f"Experience level: {strategy.get('experience_level', 'N/A')}")
            
    else:
        error = result.get('error', '')
        if 'api' in error.lower() or 'credential' in error.lower():
            print("✅ Expected error: Missing API credentials")
        elif 'llm' in error.lower() or 'groq' in error.lower():
            print("✅ Expected error: LLM setup required")
        else:
            print(f"Error: {error}")
        
except Exception as e:
    print(f"❌ search_jobs_from_cv test error: {e}")

# Test with different CV content types
print("\n=== Testing Different Content Types ===")
test_cases = [
    {
        "name": "PDF path test",
        "cv_content": "non_existent_resume.pdf",
        "cv_content_type": "pdf"
    },
    {
        "name": "Empty content test", 
        "cv_content": "",
        "cv_content_type": "text"
    },
    {
        "name": "Invalid content type test",
        "cv_content": "Some CV content",
        "cv_content_type": "invalid_type"
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n--- Test {i}: {test_case['name']} ---")
    try:
        result = search_jobs_from_cv.func(
            cv_content=test_case["cv_content"],
            cv_content_type=test_case["cv_content_type"],
            num_results=1,
            model_name=MODEL_NAME
        )
        
        print(f"Success: {result.get('success', False)}")
        
        if not result.get('success'):
            error = result.get('error', '')
            if 'empty' in error.lower() or 'not found' in error.lower() or 'invalid' in error.lower():
                print("✅ Expected error: Proper validation")
            else:
                print(f"Error: {error[:100]}...")
        else:
            print("⚠️ Unexpected success")
            
    except Exception as e:
        print(f"❌ Test {i} error: {e}")


## 8. Integration Test with Mock Data

In [ ]:
# Test integration with mock data and tool functions
print("=== Integration Test with Mock Data ===")

# Mock search result data for testing manual parser
mock_search_data = {
    'items': [
        {
            'title': 'Senior Python Developer - Netflix - San Francisco, CA - LinkedIn',
            'link': 'https://www.linkedin.com/jobs/view/3456789012',
            'snippet': 'Netflix is seeking a Senior Python Developer in San Francisco, CA. Full-time role with competitive salary $180,000-$220,000. Required: 5+ years Python, Django, PostgreSQL, AWS. Benefits: Health insurance, stock options, flexible PTO. Posted 3 days ago.'
        },
        {
            'title': 'Machine Learning Engineer | Meta | Remote - LinkedIn',
            'link': 'https://www.linkedin.com/jobs/view/2345678901',
            'snippet': 'Meta (Facebook) hiring ML Engineer for remote position. Requirements: PhD/MS in CS, Python, TensorFlow, PyTorch, 3+ years ML experience. Salary $200,000-$300,000 + equity. Posted 1 week ago.'
        },
        {
            'title': 'Frontend Developer - Airbnb - Hybrid - LinkedIn',
            'link': 'https://www.linkedin.com/jobs/view/1234567890',
            'snippet': 'Airbnb seeks Frontend Developer for hybrid role in San Francisco. React, TypeScript, Node.js required. Salary: $140,000-$180,000. 2+ years experience. Great benefits and culture. Posted 5 days ago.'
        }
    ]
}

try:
    # Test manual parser with mock data
    parser = LinkedInJobManualParser()
    jobs = parser.parse_search_results(mock_search_data)
    
    print(f"✅ Parsed {len(jobs)} jobs from mock data using manual parser")
    
    for i, job in enumerate(jobs, 1):
        print(f"\n=== Job {i} ===")
        print(f"ID: {job.get('job_id')}")
        print(f"Title: {job.get('title')}")
        print(f"Company: {job.get('company')}")
        print(f"Location: {job.get('location')}")
        print(f"Type: {job.get('job_type')}")
        print(f"Salary: {job.get('salary')}")
        print(f"Posted: {job.get('posted_date')}")
        print(f"Source: {job.get('source')}")
        
    # Test JobSchema validation with parsed data
    print("\n=== Testing JobSchema Validation with Parsed Data ===")
    for i, job in enumerate(jobs[:2], 1):  # Test first 2 jobs
        try:
            job_schema = JobSchema(
                job_id=job.get('job_id', 'None'),
                title=job.get('title', 'None'),
                location=job.get('location', 'None'),
                job_type=job.get('job_type', 'None'),
                url=job.get('url', 'None'),
                source=job.get('source', 'linkedin'),
                description=job.get('description', 'None'),
                posted_date=job.get('posted_date', 'None')
            )
            print(f"✅ Job {i} schema validation successful")
            print(f"   Validated title: {job_schema.title}")
            print(f"   Validated company: {job_schema.company_info.name}")
        except Exception as e:
            print(f"❌ Job {i} schema validation error: {e}")

    # Test tool input schema validation
    print("\n=== Testing Tool Input Schema Validation ===")
    
    # Test LinkedInJobSearchInput with mock data
    try:
        search_input = LinkedInJobSearchInput(
            keyword="Software Engineer",
            location="San Francisco, CA",
            num_results=3,
            model_name=MODEL_NAME
        )
        print(f"✅ LinkedInJobSearchInput validation successful")
        print(f"   Keyword: {search_input.keyword}")
        print(f"   Model: {search_input.model_name}")
    except Exception as e:
        print(f"❌ LinkedInJobSearchInput validation error: {e}")
    
    # Test CVParseInput with mock data
    try:
        cv_input = CVParseInput(
            content="Mock CV content for testing...",
            content_type="text",
            model_name=MODEL_NAME
        )
        print(f"✅ CVParseInput validation successful")
        print(f"   Content type: {cv_input.content_type}")
        print(f"   Content length: {len(cv_input.content)}")
    except Exception as e:
        print(f"❌ CVParseInput validation error: {e}")
            
except Exception as e:
    print(f"❌ Integration test error: {e}")

In [ ]:
# Test error handling and edge cases
print("=== Testing Error Handling ===")

# Test invalid schema data
try:
    print("\n--- Testing Invalid Schema Data ---")
    
    # Test JobSchema with defaults
    try:
        valid_job = JobSchema()  # All fields have defaults
        print(f"✅ JobSchema with defaults created: {valid_job.title}")
    except Exception as e:
        print(f"❌ JobSchema creation failed: {e}")
    
    # Test empty data handling
    print("\n--- Testing Empty Data Handling ---")
    parser = LinkedInJobManualParser()
    empty_data = {'items': []}
    empty_jobs = parser.parse_search_results(empty_data)
    print(f"✅ Empty search results handled: {len(empty_jobs)} jobs")
    
    # Test malformed data
    malformed_data = {'not_items': []}
    malformed_jobs = parser.parse_search_results(malformed_data)
    print(f"✅ Malformed data handled: {len(malformed_jobs)} jobs")
        
except Exception as e:
    print(f"❌ Error handling test error: {e}")
